# Segmenting and Clustering Neighborhoods in Toronto

In [96]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim
import folium
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Scraping Wikipedia page

In [3]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_url).text

In [4]:
soup = BeautifulSoup(wiki_page)

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
- More than one neighborhood can exist in one postal code area. If a postal code already exists in the dataframe, append further neighborhood names after a comma
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
cities = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

table = soup.find('table')
for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) > 0:
        postcode = cells[0].text.strip()
        borough = cells[1].text.strip()
        neighborhood = cells[2].text.strip()
        if borough != 'Not assigned':
            if neighborhood == 'Not assigned':
                neighborhood = borough
            # if there is such postalcode in dataframe, don't create new row
            if len(cities.loc[cities.PostalCode == postcode]) > 0:
                cities.loc[cities.PostalCode == postcode, 'Neighborhood'] += ', ' + neighborhood
            else:
                cities = cities.append({
                    'PostalCode': postcode,
                    'Borough': borough,
                    'Neighborhood': neighborhood}, ignore_index=True)

In [8]:
cities.shape

(103, 3)

In [9]:
cities.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


## Get longitude and latitude for each postal code

Geocoder doesn't work, use provided CSV data.

In [6]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
cities_geodata = pd.merge(cities, geodata, how='inner', left_on='PostalCode', right_on='Postal Code')
cities_geodata.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494


In [138]:
cities_geodata.shape

(103, 6)

## Mapping neighborhoods

In [173]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent='Coursera Capstone')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [23]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(cities_geodata['Latitude'], cities_geodata['Longitude'], cities_geodata['Borough'], cities_geodata['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get foursquare data

CLIENT_ID, CLIENT_SECRET and VERSION are stored in a secrets.json file, not committed to the repo
```json
{
	"CLIENT_ID": "your_client_id",
	"CLIENT_SECRET": "your_client_secret",
	"VERSION": "your_version"
}
```

In [34]:
secrets = json.load(open('secrets.json'))
CLIENT_ID = secrets['CLIENT_ID']
CLIENT_SECRET = secrets['CLIENT_SECRET']
VERSION = secrets['VERSION']

Functinon to get nearby venues by coordinates

In [175]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):    
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=cities_geodata['Neighborhood'],
                                 latitudes=cities_geodata['Latitude'],
                                 longitudes=cities_geodata['Longitude']
                                )

In [177]:
toronto_venues.head()

,PostalCode,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,M3A,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,M4A,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,M4A,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Some neighborhoods only have a handful of venues

In [194]:
venues_count = toronto_venues.groupby('Neighborhood').count()
venues_count.sort_values(by='Venue').head(10)

,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park",1,1,1,1,1,1,1
Roselawn,1,1,1,1,1,1,1
"Emery, Humberlea",1,1,1,1,1,1,1
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1,1
"Newtonbrook, Willowdale",1,1,1,1,1,1,1
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2,2
"Humber Bay, King's Mill Park, Kingsway Park South East, Mimico NE, Old Mill South, The Queensway East, Royal York South East, Sunnylea",2,2,2,2,2,2,2
Humber Summit,2,2,2,2,2,2,2
"Kingsview Village, Martin Grove Gardens, Richview Gardens, St. Phillips",2,2,2,2,2,2,2


Let's remove neighborhoods that have less than 5 venues

In [204]:
small_neighborhoods = venues_count[venues_count['Venue']<5].index.tolist()

In [215]:
for i in reversed(range(toronto_venues.shape[0])):
    if toronto_venues.loc[i].Neighborhood in small_neighborhoods:
        toronto_venues.drop(i, inplace=True)

In [218]:
len(toronto_venues['Neighborhood'].unique())

67

Only 100 neighborhoods out of 103 have venue data. Only 67 neighborhoods have more than 5 venues.

## Analyzing neighborhoods

### Find most frequent venues in each neighborhood

In [219]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add PostalCode column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

columns = toronto_onehot.columns.tolist()
columns = [columns.pop(columns.index('Neighborhood'))] + columns
columns = [columns.pop(columns.index('PostalCode'))] + columns
toronto_onehot = toronto_onehot[columns]

toronto_onehot.head()

,PostalCode,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
7,M5A,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,M5A,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,M5A,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,M5A,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,M5A,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Find mean frequency of each category in neighborhood

In [220]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.000000,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
3,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
4,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.055556,0.0,0.0,0.00,0.0,0.0,0.0


In [221]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [257]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant
1,Agincourt,Chinese Restaurant,Sandwich Place,Clothing Store,Lounge,Breakfast Spot
2,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place
3,"Alderwood, Long Branch",Pizza Place,Pharmacy,Pool,Skating Rink,Sandwich Place
4,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pharmacy,Frozen Yogurt Shop,Diner,Fast Food Restaurant


### Clustering neighborhoods

Let's use only 2 types of neighborhoods for simplicity.

In [256]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([0, 1, 1, 1, 0])

In [258]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Join two dataframes only for neighborhoods that have cluster labels.

In [259]:
toronto_merged = cities_geodata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636,0,Coffee Shop,Bakery,Park,Pub,Breakfast Spot
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763,0,Accessories Store,Athletics & Sports,Furniture / Home Store,Event Space,Miscellaneous Shop
4,M7A,Queen's Park,Queen's Park,M7A,43.662301,-79.389494,0,Coffee Shop,Park,Gym,Japanese Restaurant,Sushi Restaurant
7,M3B,North York,Don Mills North,M3B,43.745906,-79.352188,0,Café,Basketball Court,Caribbean Restaurant,Japanese Restaurant,Gym / Fitness Center
8,M4B,East York,"Woodbine Gardens, Parkview Hill",M4B,43.706397,-79.309937,1,Pizza Place,Fast Food Restaurant,Athletics & Sports,Gastropub,Intersection


Map the neighborhoods. Add neighborhood name and 5 most common venues to popup.

In [271]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# just use 3 primary colors
colors = ['red','green']

# add markers to the map
for lat, lon, poi, cluster, popup in zip(toronto_merged['Latitude'],
                                         toronto_merged['Longitude'],
                                         toronto_merged['Neighborhood'],
                                         toronto_merged['Cluster Labels'],
                                         ['<b>{}</b><br>{}<br>{}<br>{}<br>{}<br>{}'.format(hood,v1,v2,v3,v4,v5)
                                          for hood,v1,v2,v3,v4,v5 in
                                          zip(toronto_merged['Neighborhood'],
                                              toronto_merged['1st Most Common Venue'],
                                              toronto_merged['2nd Most Common Venue'],
                                              toronto_merged['3rd Most Common Venue'],
                                              toronto_merged['4th Most Common Venue'],
                                              toronto_merged['5th Most Common Venue'],
                                            )]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=folium.Popup(popup, max_width = 300),
        color=colors[cluster],
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Compare this to a population density map. The 'red' neighborhoods are closer to city center and generally correlate to high-density areas, while 'green' neighborhoods correlate with lower-density neighborhoods.

![](http://cdn.simplesite.com/i/96/ac/286541534799441046/i286541539311417005._szw1280h1280_.jpg)